In [1]:
import os
import numpy as np
import pandas as pd
import re
import math
import pdb
import datetime
import time

# Loading StudentLife dataset

In [2]:
path = './dataset/dataset' # dir to datatset

os.listdir(path)

['sms',
 'call_log',
 'sensing',
 'EMA',
 'calendar',
 'dinning',
 'app_usage',
 'education',
 'survey']

## Loading EMA data

In [16]:
''' 
Directories of interest

stress, sleep, exercise, deadlines, lighting, audio, charging

'''
path_db = './dataset' + '/rawaccfeat'

# ema
path_ema = path + '/EMA'
path_ema_response = path_ema + '/response'
path_ema_stress = path_ema_response + '/Stress'
path_ema_sleep = path_ema_response + '/Sleep'
path_ema_exercise = path_ema_response + '/Exercise'


# education
path_edu = path + '/education'
path_edu_deadlines = path_edu + '/deadlines.csv'

# sensing

path_sens = path + '/sensing'
path_sens_activity = path_sens + '/activity'
path_sens_dark = path_sens + '/dark'
path_sens_charge = path_sens + '/phonecharge'
path_sens_audio = path_sens + '/audio'

In [4]:
'''
Id's of interest are the ones that recorded: 
                                            stress, 
                                            exercise, 
                                            deadlines.
'''
regex = r'(u\d{2})'

files = os.listdir(path_ema_stress)
ids_with_stress = [re.search(regex, file).group(0) for file in files]

files = os.listdir(path_ema_exercise)
ids_with_exercise = [re.search(regex, file).group(0) for file in files]

df_deadline = pd.read_csv(path_edu_deadlines)
ids_with_deadlines = list(df_deadline['uid'])

uids = set(ids_with_stress + ids_with_exercise + ids_with_deadlines)

uids = {uid for uid in uids if uid in ids_with_deadlines and 
                               uid in ids_with_exercise  and 
                               uid in ids_with_stress}
# uids = list(uids)
len(uids)


44

In [63]:
'''
EMA definitions
'''
files = os.listdir(path_ema)
df_EMA_def = pd.read_json(path_ema + '/' + 'EMA_definition.json')
df_EMA_def
# df_EMA_def.iloc[7]['questions']

,name,questions
0,Social,"[{'options': '[1]0-4 persons, [2]5-9 persons, ..."
1,Class,"[{'options': '', 'question_id': 'course_id', '..."
2,Comment,"[{'options': '', 'question_id': 'comment', 'qu..."
3,Stress,"[{'options': '[1]A little stressed, [2]Definit..."
4,Sleep,"[{'options': '[1]<3, [2]3.5, [3]4, [4]4.5, [5]..."
5,Activity,"[{'options': '[1]0-10%, [2]11-25%, [3]26-50%, ..."
6,Behavior,[{'options': '(Not at all) 1 2 3 4 5 (Extremel...
7,Exercise,"[{'options': '(Yes) 1 2 (No)', 'question_id': ..."
8,Study Spaces,"[{'options': '', 'question_id': 'place', 'ques..."
9,Do Campbell's jokes suck?,"[{'options': '[1]Great, [2]OK, [3]Terrible, ',..."


In [9]:
'''
Loading stress files
'''
files = os.listdir(path_ema_stress)
df_s = pd.read_json(path_ema_stress + '/' + files[0])
df_s

,level,location,null,resp_time
0,3.0,"43.70712148268646,-72.29087061546424",NaN,2013-03-30 22:06:35
1,NaN,NaN,1,2013-03-24 10:45:11
2,NaN,NaN,1,2013-03-24 10:37:06
3,NaN,NaN,1,2013-03-24 10:37:09
4,NaN,NaN,"43.70633605,-72.28701846",2013-03-24 09:53:34
5,NaN,NaN,"43.70633605,-72.28701846",2013-03-24 10:37:08
6,3.0,"43.70286921804329,-72.29096447310383",NaN,2013-03-29 22:01:34
7,3.0,"43.70697950348372,-72.29051295213978",NaN,2013-03-29 06:01:14
8,3.0,"43.70711283219849,-72.29060053949401",NaN,2013-03-31 06:52:53
9,4.0,"43.70700745335506,-72.29084036869996",NaN,2013-04-01 06:35:30


In [10]:
'''
Loading exercise files
'''

files = os.listdir(path_ema_exercise)
ids_with_exercise 

df_EMA_exercise_sample = pd.read_json(path_ema_exercise + '/' + files[0])
list(df_EMA_exercise_sample['resp_time'][:].astype(np.int64) // 10 ** 9)


[1364877836,
 1364956140,
 1365040889,
 1365214552,
 1365113670,
 1365127225,
 1365295894,
 1365384971,
 1365460159,
 1366000498,
 1365551798,
 1365728170,
 1365638857,
 1366165456,
 1366344049,
 1366511491,
 1366790350,
 1367020814,
 1367971349,
 1369685157]

In [11]:
'''
Loading activity dataset
'''

files = os.listdir(path_sens_activity)

df_s = pd.read_csv(path_sens_activity + '/' + files[0])
df_s

,timestamp,activity inference
0,1364356932,0
1,1364356935,0
2,1364356937,0
3,1364356940,0
4,1364356943,0
5,1364356945,0
6,1364356948,0
7,1364356950,0
8,1364356953,0
9,1364356955,0


In [12]:
'''
Loading light dataset
'''

files = os.listdir(path_sens_dark)

df_s = pd.read_csv(path_sens_dark + '/' + files[0])
df_s


,start,end
0,1364420234,1364424455
1,1364490388,1364496969
2,1364497018,1364501109
3,1364510648,1364516851
4,1364519950,1364525144
5,1364528070,1364532858
6,1364533424,1364560938
7,1364561284,1364565836
8,1364569987,1364574162
9,1364574650,1364580257


In [13]:
# files = os.listdir(path_db)
# conn = sqlite3.connect(path_db + '/' + files[0])
# c =  conn.cursor()
# c.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(c.fetchall())

In [14]:
# c.execute("SELECT * FROM accfeat;")
# row = c.fetchall()
# conn.commit()

In [64]:
'''
Creating Dataframe

uid, timeofday, stress, sleep, time to deadline, #of deadlines, moving, silent,
dark, charging
'''
cols = ['uid', 'time of day', 'stress', 'sleep', 'time to deadline', '# of deadlines',
       'moving', 'silent', 'dark', 'charging']

# data_df = pd.DataFrame([[0] * len(cols)] ,columns=cols)
data_df = pd.DataFrame()
val_missing = -1
for uid in uids:
    # getting stress value
    files = os.listdir(path_ema_stress)
    idx = [i for i in range(len(files)) if uid in files[i]][-1]
    
    df_stress = pd.read_json(path_ema_stress + '/' + files[idx])
    if df_stress.empty:
        continue
    # sleep file
    
    files = os.listdir(path_ema_sleep)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1]
    df_sleep = pd.read_json(path_ema_sleep + '/' + files[t_idx])
    
    if df_sleep.empty:
        continue
    # exercise file
    
    files = os.listdir(path_ema_exercise)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1]
    df_exercise = pd.read_json(path_ema_exercise + '/' + files[t_idx])
    
    if df_exercise.empty:
        continue
    # deadline file

    df_deadline = pd.read_csv(path_edu_deadlines)

    deadlines = df_deadline[df_deadline['uid'] == uid]

    t_deadlines = deadlines.columns.values[1:]

    m = lambda d: datetime.datetime.strptime(d, "%Y-%m-%d")

    t_deadlines = list(map(m, t_deadlines))

    # activity files
    files = os.listdir(path_sens_activity)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1]

    df_activity = pd.read_csv(path_sens_activity + '/' + files[t_idx])
    
    # audio files
    files = os.listdir(path_sens_audio)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1] 
    df_audio = pd.read_csv(path_sens_audio + '/' + files[t_idx])
    
    # dark
    files = os.listdir(path_sens_dark)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1]
    df_dark = pd.read_csv(path_sens_dark + '/' + files[t_idx])

    # charging
    
    files = os.listdir(path_sens_charge)
    t_idx = [i for i in range(len(files)) if uid in files[i]][-1]    
    df_charging = pd.read_csv(path_sens_charge + '/' + files[t_idx])
    
    
    for i in range(len(df_stress)):
        if (math.isnan(df_stress.iloc[i]['level'])):
            continue
        
        t = {'uid': 0, 'time of day': 0, 'stress': 0, 'sleep': 0, 'exercise': 0, 'time to deadline': 0, '# of deadlines': 0,
       'moving': 0, 'silent': 0, 'dark': 0, 'charging': 0}
        
        stress_time = df_stress.iloc[i]['resp_time']
        stress_dt = datetime.datetime.strptime(stress_time.strftime("%Y-%m-%d"), "%Y-%m-%d") #fromtimestamp(stress_time.value // 10 ** 9)

        t['uid'] = uid

        t['stress'] = df_stress.iloc[i]['level']
        
        # getting timeofday
        
        h = stress_time.hour
        t['time of day'] = h // 6 # 0 - 6 : 0, 6 - 12: 1, 12 - 18: 2, 18 - 24: 3
        
        # sleep
        
        t_idx = [i for i in range(len(df_sleep['resp_time'])) if df_sleep['resp_time'].iloc[i].day == stress_time.day and 
                df_sleep['resp_time'].iloc[i].month == stress_time.month]
        if t_idx == []:
            continue
        
        t['sleep'] = df_sleep['hour'][t_idx[-1]]
        
        # Exercise
        vals = df_exercise[(df_exercise['resp_time'].apply(lambda dt: dt.day) == stress_time.day) & 
                           (df_exercise['resp_time'].apply(lambda dt: dt.month) == stress_time.month)]
        if (len(vals) > 0):
            t['exercise'] = vals.iloc[0]['exercise']
        else:
            continue
        
        # time to deadline (taking closest next deadline) 
        # find earliest time to deadline & count number of deadlines starting from response time of stress
        c = 0
        c_min = float("inf")
        min_idx = 0
        for j in range(len(t_deadlines)):
            if (len(deadlines[t_deadlines[j].strftime("%Y-%m-%d")]) == 0 or
                math.isnan(deadlines[t_deadlines[j].strftime("%Y-%m-%d")].iloc[0]) or 
                deadlines[t_deadlines[j].strftime("%Y-%m-%d")].iloc[0] == 0):
                continue
            if ((t_deadlines[j] - stress_dt).days >= 0 and  (t_deadlines[j] - stress_dt).days < c_min):
                c_min = (t_deadlines[j] - stress_dt).days
                min_idx = j
            if (deadlines[t_deadlines[j].strftime("%Y-%m-%d")].iloc[0] > 0):
                c += deadlines[t_deadlines[j].strftime("%Y-%m-%d")].iloc[0]
        
        # skip instances where no deadline is available
        if (c_min == float("inf")):
            continue
        
        t['time to deadline'] = c_min
        t['# of deadlines'] = c

        if (math.isnan(c)):
            pdb.set_trace()
        # moving or not

        stress_ut = stress_time.value // 10 ** 9
        
    
        vals = df_activity[df_activity['timestamp'] >= stress_ut]
        if (len(vals) > 0):
            t['moving'] = vals.iloc[0][' activity inference'] != 0
        else:
            t['moving'] = val_missing
        

        vals = df_audio[stress_ut <= df_audio['timestamp']]
        if (len(vals) > 0):
            t['silent'] = vals.iloc[0][' audio inference'] != 0
        else:
            t['silent'] = val_missing

        
        t['dark'] = len(df_dark[(df_dark['start'] <= stress_ut) & (df_dark['end'] >= stress_ut)]) != 0
    

        t['charging'] = len(df_charging[(df_charging['start'] <= stress_ut) & (df_charging['end'] >= stress_ut)]) != 0
        
        data_df = data_df.append(t, ignore_index=True)


In [65]:
 data_df

,# of deadlines,charging,dark,exercise,moving,silent,sleep,stress,time of day,time to deadline,uid
0,13.0,0.0,1.0,5.0,0.0,1.0,9.0,1.0,0.0,5.0,u15
1,13.0,0.0,0.0,5.0,0.0,0.0,9.0,1.0,0.0,5.0,u15
2,12.0,0.0,0.0,2.0,1.0,1.0,5.0,4.0,0.0,3.0,u31
3,20.0,1.0,1.0,5.0,0.0,0.0,7.0,4.0,1.0,5.0,u07
4,20.0,0.0,0.0,1.0,0.0,0.0,7.0,5.0,1.0,0.0,u07
5,20.0,0.0,0.0,1.0,0.0,0.0,7.0,4.0,0.0,0.0,u07
6,20.0,1.0,1.0,5.0,0.0,0.0,7.0,4.0,1.0,5.0,u07
7,20.0,0.0,1.0,5.0,0.0,1.0,7.0,4.0,2.0,5.0,u07
8,20.0,1.0,1.0,3.0,0.0,0.0,8.0,4.0,1.0,4.0,u07
9,20.0,0.0,1.0,3.0,0.0,1.0,8.0,1.0,3.0,4.0,u07


## Random Forest

In [66]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble        import RandomForestClassifier 
from sklearn.model_selection import cross_val_score

In [67]:
cols = data_df.columns.values
train_cols = [col for col in cols if (col != 'stress') and (col != 'uid')]

train_df = data_df[:][train_cols]
label_df = data_df[:]['stress']


In [68]:
train_df.head()

,# of deadlines,charging,dark,exercise,moving,silent,sleep,time of day,time to deadline
0,13.0,0.0,1.0,5.0,0.0,1.0,9.0,0.0,5.0
1,13.0,0.0,0.0,5.0,0.0,0.0,9.0,0.0,5.0
2,12.0,0.0,0.0,2.0,1.0,1.0,5.0,0.0,3.0
3,20.0,1.0,1.0,5.0,0.0,0.0,7.0,1.0,5.0
4,20.0,0.0,0.0,1.0,0.0,0.0,7.0,1.0,0.0


In [69]:
label_df.head()

0    1.0
1    1.0
2    4.0
3    4.0
4    5.0
Name: stress, dtype: float64

In [70]:
train_data = train_df.values
label_data = label_df.values

In [71]:
np.where(np.isinf(train_data))

(array([], dtype=int64), array([], dtype=int64))

In [72]:
np.where(np.isinf(label_data))

(array([], dtype=int64),)

In [73]:
train_data.sum(0)

array([ 18034.,    164.,    320.,   1182.,     70.,    232.,   5026.,
          883.,   1589.])

In [75]:
clf = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(clf, train_data, label_data, cv=5)
scores

array([ 0.27083333,  0.38461538,  0.25352113,  0.12676056,  0.26241135])